CONTENTS LIST:
- Creating a loyalty flag
- Creating a spending flag
- Creating a frequency flag

#01 Import libraries & data

In [3]:
import pandas as pd
import numpy as np
import os

In [7]:
#Define pathname
path = '/Users/gingermoore/Documents/04-2025 Instacart Basket Analysis'

In [9]:
#Import data
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'new_ords_prods_merge.pkl'))

#02 Find aggregated mean of orders for entire dataframe

In [12]:
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,orders_chronological,orders_day_of_week,order_hour_of_day,days_since_prior_order,is_new,add_to_cart_order,reordered,exists,price_range,busiest_days,busiest_period_of_day
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,False,5,0,both,Mid-range product,Regularly busy,Most orders
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,False,1,1,both,Mid-range product,Regularly busy,Average orders
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,False,20,0,both,Mid-range product,Busiest days,Average orders
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,True,10,0,both,Mid-range product,Least busy days,Most orders
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,False,11,1,both,Mid-range product,Least busy days,Average orders


In [29]:
ords_prods_merge.groupby('department_id').agg({'orders_chronological': ['mean']})

,orders_chronological
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


The mean order_number is higher in the whole dataset than it was for the subset. This means there were some higher values included in the full dataframe that were not included in our subset of 1000000 rows.

#03 Creating a loyalty flag

In [34]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['orders_chronological'].transform('max')

In [36]:
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,orders_chronological,orders_day_of_week,order_hour_of_day,days_since_prior_order,is_new,add_to_cart_order,reordered,exists,price_range,busiest_days,busiest_period_of_day,max_order
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,False,5,0,both,Mid-range product,Regularly busy,Most orders,32
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,False,1,1,both,Mid-range product,Regularly busy,Average orders,32
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,False,20,0,both,Mid-range product,Busiest days,Average orders,5
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,True,10,0,both,Mid-range product,Least busy days,Most orders,3
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,False,11,1,both,Mid-range product,Least busy days,Average orders,3


In [45]:
#Setting conditions

In [38]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [40]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [42]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [47]:
ords_prods_merge['loyalty_flag'].value_counts()

loyalty_flag
Regular customer    15874128
Loyal customer      10282763
New customer         6242841
Name: count, dtype: int64

In [49]:
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices           
                      mean  min   max
loyalty_flag                         
Loyal customer    7.773575  1.0  25.0
New customer      7.801206  1.0  25.0
Regular customer  7.798262  1.0  25.0

Based on these statistics, it does not appear that there is much difference in the spending habits of these customer categories. Unless I did it wrong, but I don't think I did.

#04 Creating a spending flag

In [53]:
ords_prods_merge['avg_order_total'] = ords_prods_merge.groupby(['user_id'])['prices'].transform('mean')

In [55]:
ords_prods_merge.head(15)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,orders_chronological,orders_day_of_week,order_hour_of_day,...,is_new,add_to_cart_order,reordered,exists,price_range,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_order_total
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,False,5,0,both,Mid-range product,Regularly busy,Most orders,32,Regular customer,6.935811
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,False,1,1,both,Mid-range product,Regularly busy,Average orders,32,Regular customer,6.935811
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,False,20,0,both,Mid-range product,Busiest days,Average orders,5,New customer,7.930208
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,True,10,0,both,Mid-range product,Least busy days,Most orders,3,New customer,4.972414
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,False,11,1,both,Mid-range product,Least busy days,Average orders,3,New customer,4.972414
5,1,Chocolate Sandwich Cookies,61,19,5.8,1701441,777,16,1,7,...,False,7,0,both,Mid-range product,Busiest days,Average orders,26,Regular customer,6.935398
6,1,Chocolate Sandwich Cookies,61,19,5.8,1871483,825,3,2,14,...,False,2,0,both,Mid-range product,Regularly busy,Most orders,9,New customer,5.957576
7,1,Chocolate Sandwich Cookies,61,19,5.8,1290456,910,12,3,10,...,False,1,0,both,Mid-range product,Least busy days,Most orders,12,Regular customer,6.680000
8,1,Chocolate Sandwich Cookies,61,19,5.8,369558,1052,10,1,20,...,False,1,0,both,Mid-range product,Busiest days,Average orders,20,Regular customer,7.162500
9,1,Chocolate Sandwich Cookies,61,19,5.8,589712,1052,15,1,12,...,False,2,1,both,Mid-range product,Busiest days,Most orders,20,Regular customer,7.162500


In [57]:
#setting conditions
ords_prods_merge.loc[ords_prods_merge['avg_order_total'] > 10, 'spending_flag'] = 'Low spender'

In [61]:
ords_prods_merge.loc[ords_prods_merge['avg_order_total'] <= 10, 'spending_flag'] = 'High spender'

In [63]:
ords_prods_merge['spending_flag'].value_counts()

spending_flag
High spender    32280394
Low spender       119338
Name: count, dtype: int64

#05 Creating customer frequncy flag

In [66]:
ords_prods_merge['median_days'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform('median')

In [68]:
ords_prods_merge.head(15)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,orders_chronological,orders_day_of_week,order_hour_of_day,...,reordered,exists,price_range,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_order_total,spending_flag,median_days
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,0,both,Mid-range product,Regularly busy,Most orders,32,Regular customer,6.935811,High spender,8.0
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,1,both,Mid-range product,Regularly busy,Average orders,32,Regular customer,6.935811,High spender,8.0
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,0,both,Mid-range product,Busiest days,Average orders,5,New customer,7.930208,High spender,8.0
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,0,both,Mid-range product,Least busy days,Most orders,3,New customer,4.972414,High spender,9.0
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,1,both,Mid-range product,Least busy days,Average orders,3,New customer,4.972414,High spender,9.0
5,1,Chocolate Sandwich Cookies,61,19,5.8,1701441,777,16,1,7,...,0,both,Mid-range product,Busiest days,Average orders,26,Regular customer,6.935398,High spender,11.0
6,1,Chocolate Sandwich Cookies,61,19,5.8,1871483,825,3,2,14,...,0,both,Mid-range product,Regularly busy,Most orders,9,New customer,5.957576,High spender,20.0
7,1,Chocolate Sandwich Cookies,61,19,5.8,1290456,910,12,3,10,...,0,both,Mid-range product,Least busy days,Most orders,12,Regular customer,6.680000,High spender,6.0
8,1,Chocolate Sandwich Cookies,61,19,5.8,369558,1052,10,1,20,...,0,both,Mid-range product,Busiest days,Average orders,20,Regular customer,7.162500,High spender,10.0
9,1,Chocolate Sandwich Cookies,61,19,5.8,589712,1052,15,1,12,...,1,both,Mid-range product,Busiest days,Most orders,20,Regular customer,7.162500,High spender,10.0


In [70]:
#setting conditions
ords_prods_merge.loc[ords_prods_merge['median_days'] < 20, 'frequency_flag'] = 'Non-frequent customer'

In [72]:
ords_prods_merge.loc[(ords_prods_merge['median_days'] < 10) & (ords_prods_merge['median_days'] >= 20), 'frequency_flag'] = 'Regular customer'

In [74]:
ords_prods_merge.loc[ords_prods_merge['median_days'] >= 10, 'frequency_flag'] = 'Frequent customer'

In [76]:
#checking frequency of frequency flag (lol)

In [78]:
ords_prods_merge['frequency_flag'].value_counts()

frequency_flag
Non-frequent customer    20285184
Frequent customer        12114543
Name: count, dtype: int64

In [86]:
#Dropping exists column as it is not necessary now that dataframes were merged
df_ords_prods_flags = ords_prods_merge.drop(columns = 'exists')

In [88]:
#checking new df head
df_ords_prods_flags.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,orders_chronological,orders_day_of_week,order_hour_of_day,...,reordered,price_range,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_order_total,spending_flag,median_days,frequency_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,0,Mid-range product,Regularly busy,Most orders,32,Regular customer,6.935811,High spender,8.0,Non-frequent customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,1,Mid-range product,Regularly busy,Average orders,32,Regular customer,6.935811,High spender,8.0,Non-frequent customer
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,0,Mid-range product,Busiest days,Average orders,5,New customer,7.930208,High spender,8.0,Non-frequent customer
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,0,Mid-range product,Least busy days,Most orders,3,New customer,4.972414,High spender,9.0,Non-frequent customer
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,1,Mid-range product,Least busy days,Average orders,3,New customer,4.972414,High spender,9.0,Non-frequent customer


In [90]:
#Exporting data
df_ords_prods_flags.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_w_flags.pkl'))